In [ ]:

import time
from pathlib import Path
from datetime import datetime
import SimpleITK as sitk
from monai.utils import set_determinism
import math
import torch
from torch.utils.data import random_split, DataLoader
import monai
import gdown
import pandas as pd
import torchio as tio
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset,Dataset,PersistentDataset, list_data_collate, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
from pytorch_lightning.loggers import WandbLogger

sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()

from datetime import datetime
import os
import tempfile
from glob import glob

from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

import torch.nn as nn
import torch.nn.functional as F


sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()
import preprocessing.transformsForMain as transformsForMain
import preprocessing.Nyul_preprocessing as Nyul_preprocessing
import preprocessing.manageMetaData as manageMetaData
import dataManag.utils.dataUtils as dataUtils

print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz")

#wandb_logger = WandbLogger(project="picai", name = "baseline")


aaa
zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz


In [2]:
df= manageMetaData.load_df_only_full()


Possible incompatible factory load:
Running itk version :
itk version 5.2.1
Loaded factory version:
itk version 5.3.0
Loading factory:
/home/sliceruser/Slicer/bin/../lib/Slicer-5.0/ITKFactories/libMRMLIDIOPlugin.so




10294
11154
11447
11471


In [4]:

### Define Data Handling
class PiCaiDataModule(pl.LightningDataModule):
    def __init__(self,trainSizePercent,batch_size,num_workers,drop_last,df):
        super().__init__()
        self.batch_size = batch_size
        self.df = df
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.train_set = None
        self.val_set = None
        self.test_set = None  
        self.trainSizePercent =trainSizePercent
        self.train_files = None
        self.val_files= None
        self.test_files= None
        self.train_ds = None
        self.val_ds= None
        self.test_ds= None        
        self.subjects= None        

    def splitDataSet(self,patList, trainSizePercent):
        """
        test train validation split
        TODO(balance sets)
        """
        totalLen=len(patList)
        train_test_split( patList  )
        numTrain= math.ceil(trainSizePercent*totalLen)
        numTestAndVal=totalLen-numTrain
        numTest=math.ceil(numTestAndVal*0.5)
        numVal= numTestAndVal-numTest

        # valid_set,test_set = torch.utils.data.random_split(test_and_val_set, [math. ceil(0.5), 0.5])
        print('Train data set:', numTrain)
        print('Test data set:',numTest)
        print('Valid data set:', numVal)
        return torch.utils.data.random_split(patList, [numTrain,numVal,numTest])

    def get_preprocessing_transform(self):
        preprocess = tio.Compose([
            tio.RescaleIntensity((-1, 1)),
            tio.CropOrPad(self.get_max_shape(self.subjects + self.test_subjects)),
            tio.EnsureShapeMultiple(8),  # for the U-Net
            tio.OneHot(),
        ])
        return preprocess
    
    def get_augmentation_transform(self):
        augment = tio.Compose([
            tio.RandomAffine(),
            tio.RandomGamma(p=0.5),
            tio.RandomNoise(p=0.5),
            tio.RandomMotion(p=0.1),
            tio.RandomBiasField(p=0.25),
        ])
        return augment

    def setup(self, stage=None):
        set_determinism(seed=0)
        self.subjects = list(map(lambda row: manageMetaData.getMonaiSubjectDataFromDataFrame(row[1])   , list(df.iterrows())))
        train_set, valid_set,test_set = self.splitDataSet(self.subjects , self.trainSizePercent)
        self.train_subjects = train_set
        self.val_subjects = valid_set
        self.test_subjects = test_set
        train_transforms=transformsForMain.get_train_transforms()
        val_transforms= transformsForMain.get_val_transforms()
        self.train_ds =  Dataset(data=self.train_subjects, transform=train_transforms)
        self.val_ds=     Dataset(data=self.val_subjects, transform=val_transforms)
        self.test_ds=    Dataset(data=self.test_subjects, transform=val_transforms)    

    def train_dataloader(self):
        return DataLoader(self.train_ds, self.batch_size, drop_last=self.drop_last)

    def val_dataloader(self):
        return DataLoader(self.val_ds, self.batch_size, drop_last=self.drop_last)

    def test_dataloader(self):
        return DataLoader(self.test_ds, self.batch_size, drop_last=self.drop_last)#num_workers=self.num_workers,

data = PiCaiDataModule(
    df= df,
    batch_size=1,#TODO(batc size determined by lightning)
    trainSizePercent=0.7,
    num_workers=1,
    drop_last=True
)
data.prepare_data()
data.setup()

class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer
    
    def prepare_batch(self, batch):
        return batch['t2w'], batch['label']
    
    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss


unet = monai.networks.nets.UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    #channels=(16, 16, 16, 16, 16),
    strides=(2, 2, 2, 2),
    num_res_units=1,
)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | net       | UNet       | 2.4 M 
1 | criterion | DiceCELoss | 0     
-----------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.792     Total estimated model params size (MB)


Train data set: 152
Test data set: 32
Valid data set: 32
Training started at 2022-06-13 14:55:48.376135
Train data set: 152
Test data set: 32
Valid data set: 32


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Training duration: 0:03:06.009274


In [ ]:
model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)
trainer = pl.Trainer(
    #accelerator="cpu", #TODO(remove)
    max_epochs=3,
    #gpus=1,
    #precision=16, #TODO(unhash)
    callbacks=[early_stopping],
    #logger=wandb_logger,
    accelerator='gpu',
    devices=1
)
trainer.logger._default_hp_metric = False


start = datetime.now()
print('Training started at', start)
trainer.fit(model=model, datamodule=data)
print('Training duration:', datetime.now() - start)